In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e3/sample_submission.csv
/kaggle/input/playground-series-s5e3/train.csv
/kaggle/input/playground-series-s5e3/test.csv


In [6]:
train_data = pd.read_csv("/kaggle/input/playground-series-s5e3/train.csv")

test_data = pd.read_csv("/kaggle/input/playground-series-s5e3/test.csv")


In [7]:
train_data.describe


<bound method NDFrame.describe of         id  day  pressure  maxtemp  temparature  mintemp  dewpoint  humidity  \
0        0    1    1017.4     21.2         20.6     19.9      19.4      87.0   
1        1    2    1019.5     16.2         16.9     15.8      15.4      95.0   
2        2    3    1024.1     19.4         16.1     14.6       9.3      75.0   
3        3    4    1013.4     18.1         17.8     16.9      16.8      95.0   
4        4    5    1021.8     21.3         18.4     15.2       9.6      52.0   
...    ...  ...       ...      ...          ...      ...       ...       ...   
2185  2185  361    1014.6     23.2         20.6     19.1      19.9      97.0   
2186  2186  362    1012.4     17.2         17.3     16.3      15.3      91.0   
2187  2187  363    1013.3     19.0         16.3     14.3      12.6      79.0   
2188  2188  364    1022.3     16.4         15.2     13.8      14.7      92.0   
2189  2189  365    1013.8     21.2         19.1     18.0      18.0      89.0   

     

In [12]:
features = list(train_data)
print(features[:-1])

['id', 'day', 'pressure', 'maxtemp', 'temparature', 'mintemp', 'dewpoint', 'humidity', 'cloud', 'sunshine', 'winddirection', 'windspeed']
